In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import time
import pickle
import math
import igraph as ig

In [2]:
df = pd.read_csv('extracted_games_attributes.csv')
#df = pd.read_csv('extracted_games_attributes_v3.csv')
#df = pd.read_csv('extracted_games_attributes_sample.csv')

#print(df)

#Preprocessing

In [3]:
print(f"Number of games originally: {len(df.index)}")

num_games_deleted = 0

for index, row in df.iterrows():
    if (row['positive'] + row['negative']) <= 500:
        df.drop(index, inplace=True)
        num_games_deleted += 1

print(f"Number of games deleted: {num_games_deleted}")
print(f"Number of games now: {len(df.index)}")

app_id_list = df['appid'].tolist()
app_id_np_arr = np.array(app_id_list)
with open('app_id_list.npy', 'wb') as f:
    np.save(f,app_id_np_arr)

Number of games originally: 3399
Number of games deleted: 1704
Number of games now: 1695


#Binary Matrix Creation

In [4]:
# Create the binary matrix for 'Genre' attribute -----
all_genre_set = set()

for genres in df['genre']:
    if not pd.isna(genres):
        all_genre_set.update(genres.replace(" ", "").split(','))

sorted_genre_list = list(sorted(all_genre_set))
genre_dict = dict.fromkeys(sorted_genre_list)
genre_dict.update((k, i) for i, k in enumerate(genre_dict))
genre_attr_mtx = np.zeros((len(df),len(genre_dict)), int)

for index, genres in enumerate(df['genre']):
    if not pd.isna(genres):
        genres_list = genres.replace(" ", "").split(',')
        for genre in genres_list:
            genre_attr_mtx[index][genre_dict.get(genre)] = 1

# test_genres = genre_attr_mtx[len(genre_attr_mtx)-1]
# for index, val in enumerate(test_genres):
#     if val == 1:
#         print(sorted_genre_list[index])

In [5]:
# Create the binary matrix for 'Tags' attribute -----
# Rows: games, Columns: tags
print("calculating tags")
all_tags_set = set()

for tags in df['tags']:
    if tags != "[]":
        tags_list = [tag_info.split(':')[0].strip() for tag_info in tags.replace("{", "").split(',')]
        all_tags_set.update(tags_list)

sorted_tags_list = list(sorted(all_tags_set))
tags_dict = dict.fromkeys(sorted_tags_list)
tags_dict.update((k, i) for i, k in enumerate(tags_dict))
tags_attr_mtx = np.zeros((len(df),len(tags_dict)), int)

for index, tags in enumerate(df['tags']):
    if tags != "[]":
        tags_list = [tag_info.split(':')[0].strip() for tag_info in tags.replace("{", "").split(',')]
        for tag in tags_list:
            tags_attr_mtx[index][tags_dict.get(tag)] = 1       

# test_tags = tags_attr_mtx[len(tags_attr_mtx)-1]
# for index, val in enumerate(test_tags):
#     if val == 1:
#         print(sorted_tags_list[index])

calculating tags


In [6]:
# # Create the binary matrix for 'Positive and Negative Reviews' attribute -----
# print("calculating reviews")
# # 6 Positive review percentage buckets: 0-19%,20-39%,40-69%,70-79%,80-94%,95-100% 
# thresholds = [19, 39, 69, 79, 94]
# review_buckets = 6

# reviews_attr_mtx = np.zeros((len(df),review_buckets), int)

# for index, row in enumerate(df.iterrows()):
#     #print(index)
#     row_index, row_data = row
#     pos = row_data['positive']
#     neg = row_data['negative']
#     #print(pos, neg, math.ceil(pos/(neg+pos)*100) )
    
#     indices = np.searchsorted(thresholds, math.ceil(pos/(neg+pos)*100))
    
#     for i in range(0,indices+1):
#         reviews_attr_mtx[index][i] = 1
#     print(indices, math.ceil(pos/(neg+pos)*100), reviews_attr_mtx[index])
# print(reviews_attr_mtx[len(reviews_attr_mtx)-1])

# Create the binary matrix for 'Positive and Negative Reviews' attribute -----
print("calculating reviews")
# Boolean is the percent of positive reviews >60%
reviews_attr_mtx = np.zeros((len(df),1), int)

for index, row in enumerate(df.iterrows()):
    #print(index)
    row_index, row_data = row
    pos = row_data['positive']
    neg = row_data['negative']
    #print(pos, neg, math.ceil(pos/(neg+pos)*100) )
    
    if math.ceil(pos/(neg+pos)*100) > 60:
        reviews_attr_mtx[index][0] = 1
    print(math.ceil(pos/(neg+pos)*100), reviews_attr_mtx[index])
print(reviews_attr_mtx[len(reviews_attr_mtx)-1])

calculating reviews
83 [1]
82 [1]
50 [0]
95 [1]
90 [1]
86 [1]
94 [1]
71 [1]
97 [1]
91 [1]
95 [1]
79 [1]
86 [1]
65 [1]
66 [1]
91 [1]
95 [1]
76 [1]
87 [1]
98 [1]
74 [1]
88 [1]
83 [1]
60 [0]
76 [1]
96 [1]
79 [1]
93 [1]
88 [1]
90 [1]
85 [1]
88 [1]
62 [1]
91 [1]
84 [1]
74 [1]
76 [1]
95 [1]
90 [1]
75 [1]
90 [1]
94 [1]
91 [1]
88 [1]
96 [1]
89 [1]
59 [0]
90 [1]
89 [1]
95 [1]
31 [0]
27 [0]
90 [1]
95 [1]
96 [1]
91 [1]
82 [1]
75 [1]
78 [1]
98 [1]
77 [1]
96 [1]
94 [1]
90 [1]
75 [1]
69 [1]
96 [1]
79 [1]
54 [0]
19 [0]
73 [1]
84 [1]
69 [1]
80 [1]
89 [1]
91 [1]
66 [1]
38 [0]
75 [1]
38 [0]
89 [1]
87 [1]
72 [1]
68 [1]
92 [1]
56 [0]
55 [0]
69 [1]
93 [1]
66 [1]
84 [1]
90 [1]
86 [1]
39 [0]
83 [1]
75 [1]
91 [1]
71 [1]
97 [1]
96 [1]
93 [1]
86 [1]
78 [1]
91 [1]
74 [1]
54 [0]
90 [1]
90 [1]
85 [1]
70 [1]
93 [1]
68 [1]
63 [1]
49 [0]
89 [1]
70 [1]
90 [1]
83 [1]
92 [1]
93 [1]
66 [1]
82 [1]
68 [1]
93 [1]
55 [0]
85 [1]
93 [1]
96 [1]
74 [1]
73 [1]
98 [1]
66 [1]
75 [1]
88 [1]
98 [1]
90 [1]
96 [1]
75 [1]
67 [1]
83 [1]


#Calculating Dice Similarity (DS)

In [7]:
# Create Dice similarity matrix for 'Genre' attribute -----
genre_ds = np.zeros((len(df),len(df)))
n_values = np.zeros(len(df))

start = time.time()

print("calculating genre_ds")

for i in range(len(genre_attr_mtx)):
    n_values[i] = np.sum(genre_attr_mtx[i])

for i in range(len(genre_attr_mtx)):
    for j in range(i, len(genre_attr_mtx)):
        if i != j:
            matching_count = np.sum((genre_attr_mtx[i] & genre_attr_mtx[j]) == 1)
            if n_values[i] + n_values[j] == 0:
                ds = 0
            else:
                ds = 2 * matching_count / (n_values[i] + n_values[j])
            genre_ds[i][j] = ds
            genre_ds[j][i] = ds
            
end = time.time()
print(f"Took {end - start} seconds")

calculating genre_ds
Took 13.20992398262024 seconds


In [8]:
# Create Dice similarity matrix for 'Tags' attribute -----
tags_ds = np.zeros((len(df),len(df)))
n_values = np.zeros(len(df))
start = time.time()
print("calculating trait_ds")

for i in range(len(tags_attr_mtx)):
    n_values[i] = np.sum(tags_attr_mtx[i])

for i in range(len(tags_attr_mtx)):
    for j in range(i, len(tags_attr_mtx)):
        if i != j:
            matching_count = np.sum((tags_attr_mtx[i] & tags_attr_mtx[j]) == 1)
            if n_values[i] + n_values[j] == 0:
                ds = 0
            else:
                ds = 2 * matching_count / (n_values[i] + n_values[j])
            tags_ds[i][j] = ds
            tags_ds[j][i] = ds

end = time.time()
print(f"Took {end - start} seconds")

calculating trait_ds
Took 16.321666717529297 seconds


In [9]:
# Create Dice similartiy matrix for 'Positive and Negative' attributes -----
review_ds = np.zeros((len(df),len(df)))
start = time.time()
print("calculating review_ds")

for i in range(len(reviews_attr_mtx)):
    for j in range(i, len(reviews_attr_mtx)):
        if i != j:
            val = reviews_attr_mtx[i][0] + reviews_attr_mtx[j][0]
            if val != 2:
                ds = 0
            else:
                ds = 1
            review_ds[i][j] = ds
            review_ds[j][i] = ds

end = time.time()
print(f"Took {end - start} seconds")

calculating review_ds
Took 1.5686986446380615 seconds


# Calculating Multiattribute Similarity (MAS)
--> using the Dice similarities

In [10]:
start = time.time()
print("calculating MAS")

MAS = np.zeros((len(df),len(df)))

ds_attr = 3

for i in range(len(df)):
    for j in range(i, len(df)):
        summation_ds = genre_ds[i][j] + tags_ds[i][j] + review_ds[i][j]
        mas_val = summation_ds / ds_attr
        MAS[i][j] = 1 - mas_val  # Inverse the normalized edge weight
        MAS[j][i] = 1 - mas_val

end = time.time()
print(f"Took {end - start} seconds")

calculating MAS
Took 2.544948101043701 seconds


In [11]:
print(MAS)

[[1.         0.46315789 0.89197531 ... 0.43809524 0.44298246 0.63063063]
 [0.46315789 1.         0.76436782 ... 0.62222222 0.46666667 0.51495726]
 [0.89197531 0.76436782 1.         ... 0.96491228 0.97701149 1.        ]
 ...
 [0.43809524 0.62222222 0.96491228 ... 1.         0.57777778 0.59770115]
 [0.44298246 0.46666667 0.97701149 ... 0.57777778 1.         0.63247863]
 [0.63063063 0.51495726 1.         ... 0.59770115 0.63247863 1.        ]]


#Network Analysis and Visualization

In [13]:
start = time.time()
#graph = nx.from_numpy_array(MAS)
print("calculating")
# plt.figure(figsize=(10,8))
# pos = nx.kamada_kawai_layout(graph)

# node_options = {"node_color": 'blue', "node_size": 15}
# edge_options = {"width": .50, "alpha": .5, "edge_color": "black"}

# print("drawing")

# nx.draw_networkx_nodes(graph, pos, **node_options)
# nx.draw_networkx_edges(graph, pos, **edge_options)

# plt.show()

graph_ig = ig.Graph.Weighted_Adjacency(MAS, mode='undirected')
graph_nx = nx.from_numpy_array(MAS)

i = graph_ig.community_infomap()
print(len(i))
# pal = ig.drawing.colors.ClusterColoringPalette(len(i))
# graph.vs['color'] = pal.get_many(i.membership)

# out_fig_name = "graph.eps"

# visual_style = {}

# # Define colors used for outdegree visualization
# colours = ['#fecc5c', '#a31a1c']

# # Set bbox and margin
# visual_style["bbox"] = (3000,3000)
# visual_style["margin"] = 17

# # Set vertex colours
# #visual_style["vertex_color"] = 'grey'

# # Set vertex size
# visual_style["vertex_size"] = 20

# # Set vertex lable size
# visual_style["vertex_label_size"] = 8

# # Don't curve the edges
# visual_style["edge_curved"] = False

# # Set the layout
# my_layout = graph.layout_fruchterman_reingold()
# visual_style["layout"] = my_layout

# # Plot the graph
# ig.plot(graph, out_fig_name, **visual_style).show()

# end = time.time()
# print(f"Took {end - start} seconds")

calculating
1


In [14]:
#print(graph[1][3])
print(ig.summary(graph_ig))
print(graph_ig.get_adjacency(attribute="weight")[0])

IGRAPH U-W- 1695 1437355 -- 
+ attr: weight (e)
None
[1.0, 0.4631578947368421, 0.8919753086419753, 0.6126126126126126, 0.6126126126126126, 0.6126126126126126, 0.4631578947368421, 0.4780701754385964, 0.4560185185185185, 0.4540023894862605, 0.4307992202729045, 0.41595441595441596, 0.5373563218390804, 0.5333333333333334, 0.47777777777777775, 0.44561403508771924, 0.37471264367816093, 0.39649122807017545, 0.5441176470588236, 0.4631578947368421, 0.36140350877192984, 0.5399610136452242, 0.41052631578947374, 0.5856950067476383, 0.45333333333333325, 0.39649122807017545, 0.44844844844844844, 0.44561403508771924, 0.3789473684210526, 0.5009746588693957, 0.513157894736842, 0.3971291866028709, 0.3391812865497076, 0.42656449553001285, 0.3587223587223587, 0.36204146730462516, 0.4725415070242657, 0.4658869395711501, 0.4956140350877193, 0.5789473684210527, 0.43157894736842106, 0.4280701754385965, 0.4631578947368421, 0.42656449553001285, 0.44834307992202727, 0.36204146730462516, 0.8140350877192982, 0.396

#Item Clustering

In [15]:
# start = time.time()
# communities = nx.community.louvain_communities(graph)
# num_communities = len(nx.community.louvain_communities(graph))
# print(num_communities)

# end = time.time()
# print(f"Took {end - start} seconds")

In [16]:
# print(communities)

In [17]:
start = time.time()
# hsv = plt.get_cmap("tab10")
# colors = hsv(np.linspace(0, 1.0, num_communities+1))

# plt.figure(figsize=(10,8))

# edge_options = {"width": .50, "alpha": .5, "edge_color": "black"}

# for community_id, nodes in enumerate(communities):
#     nx.draw_networkx_nodes(graph, pos, nodelist=list(nodes), node_color=[colors[community_id]], node_size = 15)
# nx.draw_networkx_edges(graph, pos, **edge_options)
# plt.show()

end = time.time()
print(f"Took {end - start} seconds")

Took 0.0 seconds


# Calculate Degree Centrality (DC)

In [18]:
from networkx.algorithms.centrality import degree_centrality

start = time.time()
print("calculating DC")

degree_centrality_dict = degree_centrality(graph_nx)

end = time.time()
print(f"Took {end - start} seconds")

calculating DC
Took 0.001995086669921875 seconds


In [19]:
print("Degree Centrality Dict")
print(degree_centrality_dict)

Degree Centrality Dict
{0: 1.0011806375442738, 1: 1.0011806375442738, 2: 1.0011806375442738, 3: 1.000590318772137, 4: 1.000590318772137, 5: 1.0011806375442738, 6: 1.0011806375442738, 7: 1.0011806375442738, 8: 1.0011806375442738, 9: 1.0011806375442738, 10: 1.0011806375442738, 11: 1.0011806375442738, 12: 1.0011806375442738, 13: 1.0011806375442738, 14: 1.0011806375442738, 15: 1.0011806375442738, 16: 1.0011806375442738, 17: 1.0011806375442738, 18: 1.0011806375442738, 19: 1.0011806375442738, 20: 1.0011806375442738, 21: 1.0011806375442738, 22: 1.0011806375442738, 23: 1.0011806375442738, 24: 1.0011806375442738, 25: 1.0011806375442738, 26: 1.0011806375442738, 27: 1.0011806375442738, 28: 1.0011806375442738, 29: 1.0011806375442738, 30: 1.0011806375442738, 31: 1.0011806375442738, 32: 1.0011806375442738, 33: 1.0011806375442738, 34: 1.0011806375442738, 35: 1.0011806375442738, 36: 1.0011806375442738, 37: 1.0011806375442738, 38: 1.0011806375442738, 39: 1.0011806375442738, 40: 1.0011806375442738, 41: 

# Calculate Closeness Centrality (CC)


In [20]:
# # NETWORKX IMPLEMENTATION -----

# from networkx.algorithms.centrality import closeness_centrality

# start = time.time()
# print("calculating CC")

# closeness_centrality_dict = closeness_centrality(graph_nx, distance='weight')
# max_centrality_val = max(closeness_centrality_dict.values())
# for key, value in closeness_centrality_dict.items():
#     closeness_centrality_dict[key] = value / max_centrality_val

# end = time.time()
# print(f"Took {end - start} seconds")

# ----------------------------------------

# IGRAPH IMPLEMENTATION -----

start = time.time()
print("calculating CC")

# # Load closeness_centrality_dict from pickle file instead of re-computing
# closeness_centrality_dict = {}
# with open('closeness_centrality_dict.pickle', 'rb') as handle:
#     closeness_centrality_dict = pickle.load(handle)

closeness_list = graph_ig.closeness(vertices=None, weights='weight', normalized=True)
closeness_centrality_dict = {k: v for k, v in enumerate(closeness_list)}

max_centrality_val = max(closeness_centrality_dict.values())
for key, value in closeness_centrality_dict.items():
    closeness_centrality_dict[key] = value / max_centrality_val

end = time.time()
print(f"Took {end - start} seconds")

calculating CC
Took 43.24406981468201 seconds


In [21]:
print("Closeness Centrality Dict")
print(closeness_centrality_dict)

Closeness Centrality Dict
{0: 0.8972871530974379, 1: 0.843933750096434, 2: 0.4825540958144309, 3: 0.7179919856747267, 4: 0.7179919856747267, 5: 0.7244479988925874, 6: 0.8416301250782561, 7: 0.7620132064046315, 8: 0.9327102391709454, 9: 0.8812852090379946, 10: 0.919936869512174, 11: 0.8483444382166401, 12: 0.8203929766341205, 13: 0.6761056448640443, 14: 0.8255158364034477, 15: 0.8856040180378817, 16: 0.9793294764514183, 17: 0.9331034160226754, 18: 0.7384330026143163, 19: 0.8271565379241769, 20: 0.93729743659329, 21: 0.765829653277616, 22: 0.8855363038288723, 23: 0.557895208377564, 24: 0.814692356686301, 25: 0.9516726835423268, 26: 0.8490652717054127, 27: 0.8179069722536239, 28: 0.9472727060837219, 29: 0.8549855637126114, 30: 0.8435834483055068, 31: 0.9429744223080684, 32: 0.9271090705653869, 33: 0.8693935731217478, 34: 0.8928666512230765, 35: 0.9223476763349648, 36: 0.8446302498477127, 37: 0.874444884472389, 38: 0.8381470954242611, 39: 0.7482337740482828, 40: 0.7781638215913061, 41: 0.9

In [22]:
# Save closeness centrality dict to a pickle file

with open("closeness_centrality_dict.pickle", 'wb') as handle:
    pickle.dump(closeness_centrality_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

#Calculate Betweenness Centrality (BC)

In [23]:
# # NETWORKX IMPLEMENTATION -----

# from networkx.algorithms.centrality import betweenness_centrality

# start = time.time()
# print("calculating BC")

# betweenness_centrality_dict = betweenness_centrality(graph_nx, normalized=True, weight='weight')

# end = time.time()
# print(f"Took {end - start} seconds")

# ----------------------------------------

# IGRAPH IMPLEMENTATION -----

start = time.time()
print("calculating CC")

# # Load closeness_centrality_dict from pickle file instead of re-computing
# betweenness_centrality_dict = {}
# with open('betweenness_centrality_dict.pickle', 'rb') as handle:
#     betweenness_centrality_dict = pickle.load(handle)

betweenness_list = graph_ig.betweenness(weights='weight')
betweenness_centrality_dict = {k: v for k, v in enumerate(betweenness_list)}

end = time.time()
print(f"Took {end - start} seconds")

calculating CC
Took 114.84245657920837 seconds


In [24]:
print("Betweenness Centrality Dict")
print(betweenness_centrality_dict)

Betweenness Centrality Dict
{0: 4.0, 1: 3.6666666666666665, 2: 0.0, 3: 2.3333333333333335, 4: 2.3333333333333335, 5: 370.0, 6: 10.5, 7: 0.0, 8: 35.5, 9: 349.0, 10: 145.0, 11: 33.0, 12: 0.0, 13: 0.0, 14: 1.0, 15: 226.5, 16: 1891.5, 17: 111.95000000000005, 18: 0.0, 19: 0.0, 20: 200.49999999999997, 21: 0.0, 22: 86.66666666666667, 23: 0.0, 24: 276.5, 25: 32.58333333333333, 26: 91.0, 27: 0.0, 28: 279.6833333333333, 29: 0.0, 30: 0.0, 31: 97.36666666666666, 32: 675.8333333333333, 33: 0.0, 34: 354.0, 35: 9.833333333333332, 36: 0.0, 37: 1.0, 38: 0.0, 39: 0.0, 40: 0.0, 41: 64.83333333333334, 42: 0.0, 43: 0.0, 44: 70.41666666666666, 45: 260.0, 46: 0.0, 47: 0.0, 48: 11.0, 49: 40.33333333333333, 50: 0.0, 51: 0.0, 52: 0.0, 53: 111.0, 54: 0.5, 55: 0.0, 56: 113.08333333333333, 57: 117.0, 58: 1.0, 59: 0.0, 60: 0.0, 61: 1.5, 62: 0.0, 63: 66.0, 64: 0.0, 65: 0.0, 66: 0.0, 67: 703.0, 68: 0.0, 69: 0.0, 70: 0.0, 71: 149.0, 72: 133.5, 73: 22.0, 74: 565.5, 75: 3.0, 76: 44.5, 77: 0.0, 78: 3.0, 79: 0.0, 80: 0.0,

In [25]:
# Save betweenness centrality dict to a pickle file

with open("betweenness_centrality_dict.pickle", 'wb') as handle:
    pickle.dump(betweenness_centrality_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

#Calculate Average Centrality (AC)

In [26]:
average_centrality_list = []

for i in range(len(df)):
    avg = (degree_centrality_dict[i] + closeness_centrality_dict[i] + betweenness_centrality_dict[i]) / 3
    average_centrality_list.append(avg)

print("Average Centrality List")
print(average_centrality_list)

Average Centrality List
[1.966155930213904, 1.8372603514357913, 0.4945782444529015, 1.3506385459267325, 1.3506385459267325, 123.90854287881228, 4.1142702542075105, 0.5877312813163017, 12.47796362557174, 116.96082194886075, 48.97370583568548, 11.616508358586971, 0.6071912047261314, 0.5590954274694394, 0.9422321579825739, 76.12892821852738, 631.1601700379986, 37.96142801785567, 0.5798712133861966, 0.6094457251561503, 67.47949269137918, 0.5890034302739632, 29.517794536013273, 0.5196919486406126, 92.77195766474352, 11.512062218139976, 30.950081969749892, 0.6063625365992992, 93.87726222565375, 0.6187220670856284, 0.6149213619499269, 33.103607242173005, 225.9205410138143, 0.6235247368886738, 118.63134909625579, 3.9189538824041903, 0.6152702957973288, 0.9585418406722209, 0.613109244322845, 0.583138137197519, 0.59311481971186, 22.255774111045543, 0.6124840122185983, 0.6464403107521431, 24.119608450978955, 87.28726448593194, 0.5130592607769254, 0.6310750315005506, 4.298449459200687, 14.08416127

#Calculate Ego-Focused Centralities (CEF)


In [ ]:
start = time.time()
print("Calculating Ego-Focused Centralities")

# Ego_centrality_matrix - node as the Target node (rows) x ego-focused centrality value for node i and T (columns)
ego_centrality_matrix = np.zeros((len(df),len(df)))

# # Load closeness_centrality_dict from pickle file instead of re-computing
# with open('ego_centrality_matrix.pickle', 'rb') as handle:
#     ego_centrality_matrix = pickle.load(handle)

for i in range(len(df)):
    for j in range(len(df)):
        if i != j:
          num_links_i_T = len(graph_ig.get_shortest_paths(i, to=j, weights = 'weight', output="vpath")[0]) - 1  # weight is ignored b/c we just want num of links between source and target nodes
          
          if num_links_i_T != 0:
              ego_centrality_matrix[j][i] = average_centrality_list[i] / num_links_i_T
          else:
              ego_centrality_matrix[j][i] = 0

end = time.time()
print(f"Took {end - start} seconds")

Calculating Ego-Focused Centralities


In [ ]:
print("Ego-Focused Centralities")
print(ego_centrality_matrix)

In [ ]:
# Save ego centrality matrix to a pickle file

with open("ego_centrality_matrix.pickle", 'wb') as handle:
    pickle.dump(ego_centrality_matrix, handle, protocol=pickle.HIGHEST_PROTOCOL)

#Testing the Recommendations

> All items are assigned a CEF value, and the top n items in the list are recommended to the user



In [ ]:
# Recommendations that are similar to game 0 - "Cooking Companions"
print(df.iloc[0])

In [ ]:
similarity_list = np.array(ego_centrality_matrix[0])
sort_index = np.argsort(-similarity_list)  # Descending sorted order --> largest to smallest centrality value
print(sort_index)
print()

sort_similarity = sorted(similarity_list, reverse=True)
print(sort_similarity)

In [ ]:
print(similarity_list[sort_index[0:3]])
print()
print(df.iloc[sort_index[0:3]])